In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import os
import torchvision.models as models
from torchvision import transforms
import pandas as pd
from torch.utils.data.sampler import WeightedRandomSampler
import random
import math
from cbam_resnext import cbam_resnext29_8x64d
from resnet_cbam import resnet50_cbam

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 16

In [3]:
csv = pd.read_csv('./images/dataset/0_index.csv',usecols=[0,2,4])
csv_numpy = csv.to_numpy()
#csv2 = pd.read_csv('./images/processed/0_index.csv',usecols=[0,2,4])
#csv_numpy2 = csv2.to_numpy()
#csv3 = pd.read_csv('./images/gabor_processed/0_index.csv',usecols=[0,2,4])
#csv_numpy3 = csv3.to_numpy()
#该文件可以共用

In [4]:
ss1,DD1,sD1,Ds1 = [],[],[],[]
ss2,DD2,sD2,Ds2 = [],[],[],[]
ss3,DD3,sD3,Ds3 = [],[],[],[]
for i in range(len(csv_numpy)):
    if csv_numpy[i][1] == 's/s':
        ss1.append('./images/dataset/' + csv_numpy[i][0])
        ss2.append('./images/process ed/' + csv_numpy[i][0])
        ss3.append('./images/gabor_processed/G_' + csv_numpy[i][0])
    elif csv_numpy[i][1] == 'D/D':
        DD1.append('./images/dataset/' + csv_numpy[i][0])
        DD2.append('./images/processed/' + csv_numpy[i][0])
        DD3.append('./images/gabor_processed/G_' + csv_numpy[i][0])
    elif csv_numpy[i][1] == 'D/s':
        Ds1.append('./images/dataset/' + csv_numpy[i][0])
        Ds2.append('./images/processed/' + csv_numpy[i][0])
        Ds3.append('./images/gabor_processed/G_' + csv_numpy[i][0])
    elif csv_numpy[i][1] == 's/D':
        sD1.append('./images/dataset/' + csv_numpy[i][0])
        sD2.append('./images/processed/' + csv_numpy[i][0])
        sD3.append('./images/gabor_processed/G_' + csv_numpy[i][0])
random.shuffle(ss2)
random.shuffle(DD2)
random.shuffle(Ds2)
random.shuffle(sD2)
print(len(ss2),len(DD2),len(Ds2),len(sD2))

225 188 41 31


In [5]:
train_ratio = 0.70
test_ratio = 0.30
train_set1, test_set1 = [],[]
train_set2, test_set2 = [],[]
train_set3, test_set3 = [],[]
#这四个for以后想想能不能优化一下
for i in range(len(ss1)):
    temp1 = [ss1[i],0]
    temp2 = [ss2[i],0]
    temp3 = [ss3[i],0]
    if i<int(train_ratio*len(ss1)):
        #train_set1.append(temp1)
        train_set2.append(temp2)
        #train_set3.append(temp3)
    else:
        #test_set1.append(temp1)
        test_set2.append(temp2)
        #test_set3.append(temp3)
for i in range(len(DD1)):
    temp1 = [DD1[i],1]
    temp2 = [DD2[i],1]
    temp3 = [DD3[i],1]
    if i<int(train_ratio*len(DD1)):
        #train_set1.append(temp1)
        train_set2.append(temp2)
        #train_set3.append(temp3)
    else:
        #test_set1.append(temp1)
        test_set2.append(temp2)
        #test_set3.append(temp3)
for i in range(len(Ds1)):
    temp1 = [Ds1[i],2]
    temp2 = [Ds2[i],2]
    temp3 = [Ds3[i],2]
    if i<int(train_ratio*len(Ds1)):
        #train_set1.append(temp1)
        train_set2.append(temp2)
        #train_set3.append(temp3)
    else:
        #test_set1.append(temp1)
        test_set2.append(temp2)
        #test_set3.append(temp3)
for i in range(len(sD1)):
    temp1 = [sD1[i],3]
    temp2 = [sD2[i],3]
    temp3 = [sD3[i],3]
    if i<int(train_ratio*len(sD1)):
        #train_set1.append(temp1)
        train_set2.append(temp2)
        #train_set3.append(temp3)
    else:
        #test_set1.append(temp1)
        test_set2.append(temp2)
        #test_set3.append(temp3)
train_set = train_set1+train_set2+train_set3
test_set = test_set1+test_set2+test_set3
random.shuffle(train_set)
random.shuffle(test_set)
print(len(train_set),len(test_set))

337 148


In [23]:
for data in train_set:
    print(data)

['./images/processed/WIN_20200720_14_26_30_Pro.jpg', 2]
['./images/processed/WIN_20200630_15_10_52_Pro.jpg', 1]
['./images/processed/WIN_20200630_15_35_26_Pro.jpg', 0]
['./images/processed/WIN_20200630_15_34_59_Pro.jpg', 0]
['./images/processed/WIN_20200630_15_21_11_Pro.jpg', 0]
['./images/processed/25A-s.jpg', 0]
['./images/processed/11C-s.jpg', 0]
['./images/processed/23C-s.jpg', 0]
['./images/processed/5A-s.jpg', 0]
['./images/processed/WIN_20200630_15_01_12_Pro.jpg', 1]
['./images/processed/11B.jpg', 1]
['./images/processed/22C-s.jpg', 0]
['./images/processed/WIN_20200630_15_31_55_Pro.jpg', 0]
['./images/processed/24A.jpg', 1]
['./images/processed/26A-s.jpg', 0]
['./images/processed/22B-s.jpg', 0]
['./images/processed/1A.jpg', 1]
['./images/processed/WIN_20200630_15_14_34_Pro.jpg', 1]
['./images/processed/15A-s.jpg', 0]
['./images/processed/WIN_20200703_14_43_36_Pro.jpg', 0]
['./images/processed/27B-s.jpg', 0]
['./images/processed/25B-s.jpg', 0]
['./images/processed/WIN_20200630_15

In [6]:
class MyDataset(Dataset):
    def __init__(self, dataset, train, transform=None, target_transform=None):
        imgs = []  # 内部为多个tuple，每个tuple为(图片路径，label)
        if train:
            for data in dataset:
                for i in range(3):#训练集数据复制三份
                    imgs.append((data[0],data[1]))
        else:
            for data in dataset:
                imgs.append((data[0],data[1]))            
        self.imgs = imgs
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        fn, label = self.imgs[index]
        img = Image.open(fn).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.imgs)

In [20]:
img = Image.open(ss1[0]).convert('RGB')
tR = transforms.RandomCrop(896)
tF = transforms.FiveCrop(896)
tFlist = []
for i in tF(img):
    tFlist.append(i)
a = transforms.ToTensor()(tR(img))
a.shape

torch.Size([3, 896, 896])

In [7]:
train_transforms = transforms.Compose([
    transforms.RandomCrop(896),
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])
'''test_transforms = transforms.Compose([
    transforms.Resize(896),
    transforms.Grayscale(num_output_channels=3),
    transforms.FiveCrop(224),#不用tencrop了，感觉再增加更多相似图片，没啥用
    transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
    transforms.Lambda(lambda crops: torch.stack([transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(crop) for crop in crops])),
])'''

'test_transforms = transforms.Compose([\n    transforms.Resize(896),\n    transforms.Grayscale(num_output_channels=3),\n    transforms.FiveCrop(224),#不用tencrop了，感觉再增加更多相似图片，没啥用\n    transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),\n    transforms.Lambda(lambda crops: torch.stack([transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(crop) for crop in crops])),\n])'

In [8]:
train_data = MyDataset(train_set, train=True, transform=train_transforms)
test_data = MyDataset(test_set, train=False, transform=test_transforms)

weights = []
for data,target in train_data.imgs:
    if target == 2:
        weights.append(4)
    elif target == 3:
        weights.append(5)
    else:
        weights.append(1)
sampler = WeightedRandomSampler(weights, num_samples=1440, replacement=True)

# 然后就是调用DataLoader和刚刚创建的数据集，来创建dataloader，这里提一句，loader的长度是有多少个batch，所以和batch_size有关
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, sampler=sampler, num_workers=0)
#train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
print('num_of_trainData:', len(train_loader))
print('num_of_testData:', len(test_loader))

num_of_trainData: 90
num_of_testData: 10


In [9]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, data in enumerate(train_loader):
        x, y = data
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        y_hat = model(x)
        loss = criterion(y_hat, y)
        loss.backward()
        optimizer.step()
    print('Train Epoch: {}\t Loss: {:.6f}'.format(epoch, loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            x, y = data
            #bs, nc, c, h, w = x.size()
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            y_hat = model(x)
            #y_hat_all = model(x.view(-1,c,h,w))
            #y_hat = y_hat_all.view(bs,nc,-1).mean(1)#求5个crop的平均预测结果
            test_loss += criterion(y_hat, y).item()  # sum up batch loss
            pred = y_hat.max(1, keepdim=True)[1]  # get the index of the max log-probability
            correct += pred.eq(y.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_data),
        100. * correct / len(test_data)))

In [14]:
model_cbam

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (ca): ChannelAttention(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (

In [10]:
model_cbam = resnet50_cbam(pretrained=True)

for param in model_cbam.parameters():
    param.requires_grad = False
model_cbam.fc = nn.Linear(2048,4)

model_cbam = model_cbam.to(DEVICE)

# criterion = nn.CrossEntropyLoss()
criterion = nn.CrossEntropyLoss()

# criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([
    {'params': model_cbam.fc.parameters()},
], lr=0.001)

In [12]:
model_cbam

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (ca): ChannelAttention(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (

In [11]:
for epoch in range(1, 100):
    train(model=model_cbam, device=DEVICE, train_loader=train_loader, optimizer = optimizer, epoch=epoch)
    test(model=model_cbam, device=DEVICE, test_loader=test_loader)

Train Epoch: 1	 Loss: 1.228414

Test set: Average loss: 0.0651, Accuracy: 97/148 (66%)

Train Epoch: 2	 Loss: 0.852684

Test set: Average loss: 0.0612, Accuracy: 101/148 (68%)

Train Epoch: 3	 Loss: 1.030288

Test set: Average loss: 0.0568, Accuracy: 107/148 (72%)

Train Epoch: 4	 Loss: 1.007351

Test set: Average loss: 0.0517, Accuracy: 110/148 (74%)

Train Epoch: 5	 Loss: 0.705082

Test set: Average loss: 0.0562, Accuracy: 97/148 (66%)

Train Epoch: 6	 Loss: 0.711341

Test set: Average loss: 0.0512, Accuracy: 106/148 (72%)

Train Epoch: 7	 Loss: 0.545389

Test set: Average loss: 0.0487, Accuracy: 110/148 (74%)

Train Epoch: 8	 Loss: 0.633772

Test set: Average loss: 0.0480, Accuracy: 112/148 (76%)

Train Epoch: 9	 Loss: 1.004522

Test set: Average loss: 0.0456, Accuracy: 112/148 (76%)

Train Epoch: 10	 Loss: 0.524925

Test set: Average loss: 0.0472, Accuracy: 111/148 (75%)

Train Epoch: 11	 Loss: 0.507167

Test set: Average loss: 0.0468, Accuracy: 110/148 (74%)



KeyboardInterrupt: 